<a href="https://colab.research.google.com/github/codingniket/Python-Training/blob/main/21-01-2026/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import trim, col, when, to_date, sum as spark_sum, avg, desc, rank, lit, coalesce, isnull,try_to_timestamp,regexp_extract,initcap,regexp_replace,to_timestamp,unix_timestamp,dense_rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Final").getOrCreate()

In [2]:
print("--- PHASE 1: INGESTION ---")

--- PHASE 1: INGESTION ---


In [3]:
df_raw = spark.read \
.option("header", "true") \
.option("inferSchema", "false") \
.csv("hospital_visits.csv")

df_raw.show()
df_raw.printSchema()
df_raw.count()
df_raw.describe()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+
| V100000|     P8243|     D247| Cardiology|    Hyderabad|22/01/2026 00:50:00|               NULL|               NULL|               NULL|      CANCELLED|       invalid|        Cash|              HDFC|  NULL|
| V100001|     P1663|     D185| Cardiology|       Mumbai|2026-01-11 13:47:00|2026-01-11 14:27:00|2026-01-11 14:40:00|2026-01-11 16:24:00|     REGISTERED|           757|

DataFrame[summary: string, visit_id: string, patient_id: string, doctor_id: string, department: string, hospital_city: string, checkin_time: string, consult_start_time: string, consult_end_time: string, discharge_time: string, visit_status: string, payment_amount: string, payment_mode: string, insurance_provider: string, rating: string]

In [4]:

df_raw.count()

120000

In [5]:
df_raw.isEmpty()

False

# 4

1. Many data in `consult_start_time` & `consult_start_time` are Null and date and time
are not valid format

2. `rating` there are NULL Value also present here

In [6]:
print("--- PHASE 2: CLEANING ---")

--- PHASE 2: CLEANING ---


In [7]:
# 1. Trim all string columns
df_trimmed = df_raw.select([trim(col(c)).alias(c) for c in df_raw.columns])
df_trimmed.show()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+
| V100000|     P8243|     D247| Cardiology|    Hyderabad|22/01/2026 00:50:00|               NULL|               NULL|               NULL|      CANCELLED|       invalid|        Cash|              HDFC|  NULL|
| V100001|     P1663|     D185| Cardiology|       Mumbai|2026-01-11 13:47:00|2026-01-11 14:27:00|2026-01-11 14:40:00|2026-01-11 16:24:00|     REGISTERED|           757|

In [8]:
# 2. Clean payment_amount
# Remove commas, convert to Int, handle invalid/empty as null
df_cleaned_pay = df_trimmed.withColumn(
    "payment_amount_clean",
    when(col("payment_amount") == "", None)
    .when(col("payment_amount") == "invalid", None)
    .otherwise(regexp_replace(col("payment_amount"), ",", "").cast(IntegerType()))
)

df_cleaned_pay.show()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|payment_amount_clean|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+
| V100000|     P8243|     D247| Cardiology|    Hyderabad|22/01/2026 00:50:00|               NULL|               NULL|               NULL|      CANCELLED|       invalid|        Cash|              HDFC|  NULL|                NULL|
| V100001|     P1663|     D185| Cardiology|       Mumbai|2026-01-11 13:47:00|2026-01

In [9]:
# 3. Parse time columns (Handling mixed formats)
# Formats: yyyy-MM-dd HH:mm:ss, dd/MM/yyyy HH:mm:ss, yyyy/MM/dd HH:mm:ss
def parse_mixed_time(c):
    return coalesce(
        to_timestamp(try_to_timestamp(c, lit("yyyy-MM-dd HH:mm:ss"))),
        to_timestamp(try_to_timestamp(c, lit("dd/MM/yyyy HH:mm:ss"))),
        to_timestamp(try_to_timestamp(c, lit("yyyy/MM/dd HH:mm:ss"))),
    )

time_cols = ["checkin_time", "consult_start_time", "consult_end_time", "discharge_time"]
df_parsed = df_cleaned_pay
for c in time_cols:
    df_parsed = df_parsed.withColumn(f"{c}_clean", parse_mixed_time(col(c)))

df_parsed.show()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+-------------------+------------------------+----------------------+--------------------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|payment_amount_clean| checkin_time_clean|consult_start_time_clean|consult_end_time_clean|discharge_time_clean|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+-------------------+------------------------+----------------------+--------------------+
| V100000|     P8243|     D247| Cardiology|   

In [10]:
# 4. Create validity flags
df_flagged = df_parsed.withColumn("checkin_valid", col("checkin_time_clean").isNotNull()) \
    .withColumn("consult_start_valid", col("consult_start_time_clean").isNotNull()) \
    .withColumn("consult_end_valid", col("consult_end_time_clean").isNotNull()) \
    .withColumn("discharge_valid", col("discharge_time_clean").isNotNull())

df_flagged.show()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+-------------------+------------------------+----------------------+--------------------+-------------+-------------------+-----------------+---------------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|payment_amount_clean| checkin_time_clean|consult_start_time_clean|consult_end_time_clean|discharge_time_clean|checkin_valid|consult_start_valid|consult_end_valid|discharge_valid|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+

In [11]:
print("--- PHASE 3: DERIVED TIME METRICS ---")

--- PHASE 3: DERIVED TIME METRICS ---


In [12]:
# Compute metrics in minutes using unix_timestamp
df_metrics = df_flagged.withColumn(
    "waiting_time",
    (unix_timestamp("consult_start_time_clean") - unix_timestamp("checkin_time_clean")) / 60
).withColumn(
    "consultation_time",
    (unix_timestamp("consult_end_time_clean") - unix_timestamp("consult_start_time_clean")) / 60
).withColumn(
    "total_visit_time",
    (unix_timestamp("discharge_time_clean") - unix_timestamp("checkin_time_clean")) / 60
)

df_metrics.show()

+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+------------------+------+--------------------+-------------------+------------------------+----------------------+--------------------+-------------+-------------------+-----------------+---------------+------------+-----------------+----------------+
|visit_id|patient_id|doctor_id| department|hospital_city|       checkin_time| consult_start_time|   consult_end_time|     discharge_time|   visit_status|payment_amount|payment_mode|insurance_provider|rating|payment_amount_clean| checkin_time_clean|consult_start_time_clean|consult_end_time_clean|discharge_time_clean|checkin_valid|consult_start_valid|consult_end_valid|discharge_valid|waiting_time|consultation_time|total_visit_time|
+--------+----------+---------+-----------+-------------+-------------------+-------------------+-------------------+---------------

In [13]:
# Flag negative or null values (Requirement: Negative or null values must be flagged)
# We create a simple flag column "is_metric_valid" for demonstration
df_metrics = df_metrics.withColumn(
    "has_negative_metrics",
    (col("waiting_time") < 0) | (col("consultation_time") < 0) | (col("total_visit_time") < 0)
)
df_metrics.select("visit_id", "waiting_time", "consultation_time", "total_visit_time", "has_negative_metrics").show()

+--------+------------+-----------------+----------------+--------------------+
|visit_id|waiting_time|consultation_time|total_visit_time|has_negative_metrics|
+--------+------------+-----------------+----------------+--------------------+
| V100000|        NULL|             NULL|            NULL|                NULL|
| V100001|        40.0|             13.0|           157.0|               false|
| V100002|        32.0|             44.0|           128.0|               false|
| V100003|         0.0|            -15.0|            59.0|                true|
| V100004|        24.0|             19.0|            98.0|               false|
| V100005|        23.0|             31.0|           165.0|               false|
| V100006|        39.0|            -14.0|            48.0|                true|
| V100007|        35.0|             35.0|           183.0|               false|
| V100008|        17.0|             27.0|           100.0|               false|
| V100009|        33.0|             -5.0

In [14]:
print("--- PHASE 4: DATA QUALITY AUDIT ---")

# 1. Count logic errors
audit_1 = df_metrics.filter(col("consult_start_time_clean") < col("checkin_time_clean")).count()
audit_2 = df_metrics.filter(col("consult_end_time_clean") < col("consult_start_time_clean")).count()
audit_3 = df_metrics.filter(col("discharge_time_clean") < col("consult_end_time_clean")).count()


--- PHASE 4: DATA QUALITY AUDIT ---


In [15]:

# 2. Count DISCHARGED visits with missing consult_end_time
audit_4 = df_metrics.filter(
    (col("visit_status") == "DISCHARGED") & (col("consult_end_time_clean").isNull())
).count()

# 3. Count invalid payment_amount (Where original was not null/empty but clean is null)
audit_5 = df_metrics.filter(
    (col("payment_amount") != "") &
    (col("payment_amount").isNotNull()) &
    (col("payment_amount_clean").isNull())
).count()

# 4. Count CANCELLED visits
audit_6 = df_metrics.filter(col("visit_status") == "CANCELLED").count()




In [16]:
print(f"Logic Error (Start < Checkin): {audit_1}")
print(f"Logic Error (End < Start): {audit_2}")
print(f"Logic Error (Discharge < End): {audit_3}")
print(f"Discharged with missing End Time: {audit_4}")
print(f"Invalid Payment Amounts: {audit_5}")
print(f"Cancelled Visits: {audit_6}")

Logic Error (Start < Checkin): 21359
Logic Error (End < Start): 19962
Logic Error (Discharge < End): 14793
Discharged with missing End Time: 2226
Invalid Payment Amounts: 5715
Cancelled Visits: 30075


In [17]:
print("--- PHASE 5: OPERATIONAL ANALYTICS ---")

# 1. Avg waiting time per department
df_metrics.groupBy("department").agg(avg("waiting_time").alias("avg_wait_min")).show()

# 2. Avg consultation time per doctor
df_metrics.groupBy("doctor_id").agg(avg("consultation_time").alias("avg_consult_min")).show()

# 3. Departments with longest patient queues (Count of visits or Sum of wait time)
df_metrics.groupBy("department").count().orderBy(desc("count")).show()

# 4. Cities with longest discharge delays (assuming Total Visit Time acts as proxy for delay/length)
df_metrics.groupBy("hospital_city").agg(avg("total_visit_time").alias("avg_visit_len")).orderBy(desc("avg_visit_len")).show()

# 5. Cancellation rate per department
total_per_dept = df_metrics.groupBy("department").count().withColumnRenamed("count", "total")
cancelled_per_dept = df_metrics.filter(col("visit_status") == "CANCELLED").groupBy("department").count().withColumnRenamed("count", "cancelled")

# Join and calculate rate
total_per_dept.join(cancelled_per_dept, "department", "left") \
    .withColumn("cancellation_rate", coalesce(col("cancelled"), lit(0)) / col("total")) \
    .show()

--- PHASE 5: OPERATIONAL ANALYTICS ---
+-----------+------------------+
| department|      avg_wait_min|
+-----------+------------------+
|    General| 14.87842247911942|
|  Neurology| 15.09793217535153|
|Dermatology|14.767209148889378|
| Cardiology|15.013323182337267|
| Pediatrics|14.974878801234023|
|Orthopedics|15.017192924450322|
+-----------+------------------+

+---------+------------------+
|doctor_id|   avg_consult_min|
+---------+------------------+
|     D738|             23.36|
|     D607|22.469026548672566|
|     D634|21.605042016806724|
|     D907| 21.31451612903226|
|     D736|23.752475247524753|
|     D765|19.855670103092784|
|     D723|21.650406504065042|
|     D565|22.386138613861387|
|     D787| 21.51685393258427|
|     D174|25.838461538461537|
|     D448| 23.99047619047619|
|     D105|20.954128440366972|
|     D935| 23.00952380952381|
|     D955|20.041666666666668|
|     D636|23.547826086956523|
|     D657| 23.21212121212121|
|     D248| 23.78301886792453|
|     D900

In [19]:
print("--- PHASE 6: WINDOW FUNCTIONS ---")

# 1. Rank doctors by Shortest consultation time & Highest ratings
# Note: Rating is string in source, need to cast to float.
df_metrics = df_metrics.withColumn("rating_float", col("rating").cast("float"))

w_doc = Window.orderBy(col("consultation_time").asc(), col("rating_float").desc())
df_metrics.filter(col("visit_status") == "DISCHARGED") \
    .select("doctor_id", "consultation_time", "rating_float") \
    .withColumn("doc_rank", rank().over(w_doc)).show()



--- PHASE 6: WINDOW FUNCTIONS ---
+---------+-----------------+------------+--------+
|doctor_id|consultation_time|rating_float|doc_rank|
+---------+-----------------+------------+--------+
|     D224|             NULL|         5.0|       1|
|     D959|             NULL|         5.0|       1|
|     D158|             NULL|         5.0|       1|
|     D576|             NULL|         5.0|       1|
|     D831|             NULL|         5.0|       1|
|     D800|             NULL|         5.0|       1|
|     D126|             NULL|         5.0|       1|
|     D411|             NULL|         5.0|       1|
|     D329|             NULL|         5.0|       1|
|     D961|             NULL|         5.0|       1|
|     D832|             NULL|         5.0|       1|
|     D134|             NULL|         5.0|       1|
|     D878|             NULL|         5.0|       1|
|     D461|             NULL|         5.0|       1|
|     D367|             NULL|         5.0|       1|
|     D681|             NULL| 

In [20]:
# 2. Rank departments by Avg waiting time & Cancellation rate
# (Requires pre-aggregation, reusing Phase 5 logic conceptually)

# 3. Identify Top 3 efficient doctors per department (Efficiency = Shortest Consult Time for demo)
w_dept_doc = Window.partitionBy("department").orderBy(col("consultation_time").asc())

df_metrics.filter(col("visit_status") == "DISCHARGED") \
    .withColumn("rank", rank().over(w_dept_doc)) \
    .filter(col("rank") <= 3) \
    .select("department", "doctor_id", "consultation_time", "rank") \
    .show()

# 4. Bottom 3 overloaded departments (Longest Avg Wait Time)
# (Simple sort on aggregated DF from Phase 5.1)

+----------+---------+-----------------+----+
|department|doctor_id|consultation_time|rank|
+----------+---------+-----------------+----+
|Cardiology|     D720|             NULL|   1|
|Cardiology|     D616|             NULL|   1|
|Cardiology|     D384|             NULL|   1|
|Cardiology|     D238|             NULL|   1|
|Cardiology|     D648|             NULL|   1|
|Cardiology|     D820|             NULL|   1|
|Cardiology|     D249|             NULL|   1|
|Cardiology|     D220|             NULL|   1|
|Cardiology|     D659|             NULL|   1|
|Cardiology|     D996|             NULL|   1|
|Cardiology|     D922|             NULL|   1|
|Cardiology|     D834|             NULL|   1|
|Cardiology|     D647|             NULL|   1|
|Cardiology|     D807|             NULL|   1|
|Cardiology|     D658|             NULL|   1|
|Cardiology|     D848|             NULL|   1|
|Cardiology|     D519|             NULL|   1|
|Cardiology|     D576|             NULL|   1|
|Cardiology|     D938|            

In [21]:
print("--- PHASE 7: REVENUE ANALYTICS ---")

# 1. Total revenue per department
df_metrics.groupBy("department").agg(spark_sum("payment_amount_clean").alias("total_revenue")).show()

# 2. Revenue per insurance provider
df_metrics.groupBy("insurance_provider").agg(spark_sum("payment_amount_clean").alias("revenue")).show()

# 3. Average bill amount per city
df_metrics.groupBy("hospital_city").agg(avg("payment_amount_clean").alias("avg_bill")).show()

# 4. Compare insured vs non-insured revenue
# Assuming "None" or null implies non-insured
df_metrics.withColumn("is_insured",
                      when((col("insurance_provider") == "None") | (col("insurance_provider").isNull()), "Non-Insured")
                      .otherwise("Insured")) \
          .groupBy("is_insured").agg(spark_sum("payment_amount_clean").alias("revenue")).show()

--- PHASE 7: REVENUE ANALYTICS ---
+-----------+-------------+
| department|total_revenue|
+-----------+-------------+
|    General|    186426309|
|  Neurology|    185727182|
|Dermatology|    184971637|
| Cardiology|    188217091|
| Pediatrics|    185903945|
|Orthopedics|    187317569|
+-----------+-------------+

+------------------+---------+
|insurance_provider|  revenue|
+------------------+---------+
|        StarHealth|223159034|
|              None|223796193|
|              HDFC|224459133|
|             Aetna|223457749|
|             ICICI|223691624|
+------------------+---------+

+-------------+------------------+
|hospital_city|          avg_bill|
+-------------+------------------+
|    Bangalore|10105.220305284762|
|      Chennai|10138.377779185406|
|       Mumbai|10151.512569832403|
|      Kolkata|10182.544927536232|
|         Pune|10139.925238933602|
|        Delhi| 10116.51864557525|
|    Hyderabad|10123.922733932097|
+-------------+------------------+

+-----------+-----

In [22]:
print("--- PHASE 8: PERFORMANCE ENGINEERING ---")

# 1. Cache cleaned dataset
df_metrics.cache()
print(f"Dataset cached. Count: {df_metrics.count()}")

# 2. Use explain(True)
print("Explain Plan for Doctor Ranking:")
df_metrics.orderBy("consultation_time").explain(True)

# 3. Identify shuffle stages (Look for Exchange in the explain plan above)

# 4. Repartition by department
df_repartitioned = df_metrics.repartition(col("department"))

# 5. Compare execution plans
print("Explain Plan AFTER Repartitioning:")
df_repartitioned.groupBy("department").count().explain()

--- PHASE 8: PERFORMANCE ENGINEERING ---
Dataset cached. Count: 120000
Explain Plan for Doctor Ranking:
== Parsed Logical Plan ==
'Sort ['consultation_time ASC NULLS FIRST], true
+- Project [visit_id#449, patient_id#450, doctor_id#451, department#452, hospital_city#453, checkin_time#454, consult_start_time#455, consult_end_time#456, discharge_time#457, visit_status#458, payment_amount#459, payment_mode#460, insurance_provider#461, rating#462, payment_amount_clean#520, checkin_time_clean#581, consult_start_time_clean#582, consult_end_time_clean#583, discharge_time_clean#584, checkin_valid#657, consult_start_valid#658, consult_end_valid#659, discharge_valid#660, waiting_time#745, consultation_time#746, ... 3 more fields]
   +- Project [visit_id#449, patient_id#450, doctor_id#451, department#452, hospital_city#453, checkin_time#454, consult_start_time#455, consult_end_time#456, discharge_time#457, visit_status#458, payment_amount#459, payment_mode#460, insurance_provider#461, rating#462, 

In [23]:
print("--- PHASE 9: RDD ---")

# 1. Convert DISCHARGED visits to RDD
rdd_discharged = df_metrics.filter(col("visit_status") == "DISCHARGED").rdd

# 2. Compute Total hospital revenue using reduce
# Map to (payment_amount), then reduce
total_rev = rdd_discharged \
    .map(lambda row: row["payment_amount_clean"] if row["payment_amount_clean"] else 0) \
    .reduce(lambda x, y: x + y)

print(f"Total Revenue (RDD Reduce): {total_rev}")

# Visit count per city using map-reduce
city_counts = rdd_discharged \
    .map(lambda row: (row["hospital_city"], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .collect()

print(f"Visit Counts per City (RDD Map-Reduce): {city_counts}")

# 3. Explain why DataFrames are superior
print("""
Why DataFrames are superior for healthcare analytics:
1. Catalyst Optimizer: DataFrames utilize the Catalyst optimizer for logical and physical query optimization, which RDDs lack.
2. Tungsten Execution Engine: DataFrames use off-heap memory management and code generation, processing data faster than RDD python objects.
3. Structured Data Handling: Healthcare data (timestamps, currency) is structured; DataFrames enforce schema, reducing runtime type errors compared to RDDs.
""")

# Stop Spark
spark.stop()

--- PHASE 9: RDD ---
Total Revenue (RDD Reduce): 278741105
Visit Counts per City (RDD Map-Reduce): [('Pune', 4264), ('Delhi', 4285), ('Mumbai', 4210), ('Hyderabad', 4255), ('Kolkata', 4325), ('Bangalore', 4216), ('Chennai', 4319)]

Why DataFrames are superior for healthcare analytics:
1. Catalyst Optimizer: DataFrames utilize the Catalyst optimizer for logical and physical query optimization, which RDDs lack.
2. Tungsten Execution Engine: DataFrames use off-heap memory management and code generation, processing data faster than RDD python objects.
3. Structured Data Handling: Healthcare data (timestamps, currency) is structured; DataFrames enforce schema, reducing runtime type errors compared to RDDs.

